In [6]:
# Stock Weekly SVM Assignment (CS-677)
# Upload 'Processed_Stock_Data.csv' and 'NVDA.csv' manually in Colab

import pandas as pd
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score

# Upload manually
from google.colab import files
uploaded = files.upload()

# Load datasets
stock_df = pd.read_csv("Processed_Stock_Data.csv")
stock_df['Label_Num'] = stock_df['Label'].map({'Green': 1, 'Red': 0})

# Prepare training and testing sets
X_train = stock_df[stock_df['Year'].between(2020, 2022)][['mean_return', 'volatility']]
y_train = stock_df[stock_df['Year'].between(2020, 2022)]['Label_Num']
X_test = stock_df[stock_df['Year'].between(2023, 2024)][['mean_return', 'volatility']]
y_test = stock_df[stock_df['Year'].between(2023, 2024)]['Label_Num']

# Evaluate function
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)
    tn, fp, fn, tp = cm.ravel()
    tpr = tp / (tp + fn)
    tnr = tn / (tn + fp)
    return y_pred, round(acc * 100, 2), cm.tolist(), round(tpr * 100, 2), round(tnr * 100, 2)

# Train models
svm_linear = SVC(kernel='linear')
svm_linear.fit(X_train, y_train)
y_pred_linear, acc_linear, cm_linear, tpr_linear, tnr_linear = evaluate_model(svm_linear, X_test, y_test)

svm_rbf = SVC(kernel='rbf')
svm_rbf.fit(X_train, y_train)
y_pred_rbf, acc_rbf, cm_rbf, tpr_rbf, tnr_rbf = evaluate_model(svm_rbf, X_test, y_test)

svm_poly = SVC(kernel='poly', degree=2)
svm_poly.fit(X_train, y_train)
y_pred_poly, acc_poly, cm_poly, tpr_poly, tnr_poly = evaluate_model(svm_poly, X_test, y_test)

# Load NVDA weekly prices
nvda_df = pd.read_csv("NVDA.csv")
weekly_prices = nvda_df[nvda_df['Year'].between(2023, 2024)].groupby(['Year', 'Week_Number']).agg(
    Open_Price=('Open', 'first'),
    Close_Price=('Close', 'last')
).reset_index()

# Merge linear SVM predictions with weekly prices
test_df = stock_df[stock_df['Year'].between(2023, 2024)].copy()
test_df['Linear_SVM_Pred'] = y_pred_linear
test_df = test_df.merge(weekly_prices, on=['Year', 'Week_Number'], how='left')

# Trading strategy simulation
def simulate_trading(df, label_col):
    cash, shares, position = 100, 0, False
    for _, row in df.iterrows():
        if pd.isna(row['Open_Price']) or pd.isna(row['Close_Price']):
            continue
        if not position and row[label_col] == 1:
            shares = cash / row['Open_Price']
            cash = 0
            position = True
        elif position and row[label_col] == 0:
            cash = shares * row['Close_Price']
            shares = 0
            position = False
    return round(cash + (shares * row['Close_Price'] if position else 0), 2)

svm_linear_strategy_value = simulate_trading(test_df, 'Linear_SVM_Pred')
initial_open = test_df.iloc[0]['Open_Price']
final_close = test_df.iloc[-1]['Close_Price']
buy_hold_value = round((100 / initial_open) * final_close, 2)

# Summary Results
print("Linear SVM Accuracy:", acc_linear, "%")
print("Linear SVM Confusion Matrix:", cm_linear)
print("Linear SVM TPR:", tpr_linear, "%")
print("Linear SVM TNR:", tnr_linear, "%")

print("\nGaussian RBF SVM Accuracy:", acc_rbf, "%")
print("Polynomial Degree 2 SVM Accuracy:", acc_poly, "%")

print("\nLinear SVM Strategy Final Value:", svm_linear_strategy_value)
print("Buy-and-Hold Final Value:", buy_hold_value)


Saving NVDA.csv to NVDA.csv
Saving Processed_Stock_Data.csv to Processed_Stock_Data.csv
Linear SVM Accuracy: 86.67 %
Linear SVM Confusion Matrix: [[56, 3], [11, 35]]
Linear SVM TPR: 76.09 %
Linear SVM TNR: 94.92 %

Gaussian RBF SVM Accuracy: 93.33 %
Polynomial Degree 2 SVM Accuracy: 87.62 %

Linear SVM Strategy Final Value: 1390.12
Buy-and-Hold Final Value: 925.86
